In [1]:
from FileTools import FileTools
from pathlib import Path
import zipfile
import random
import json
import time
import os

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import math
import re

from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
# authenticate with API

currWorkDir = os.getcwd()
userDir = Path.home()
keyPath = f"{userDir}\\PYC\\ADMIN\\kaggle.json"

with open( keyPath, 'r' ) as f: keyDict = json.load( f )
userTitle, keyTitle = keyDict.keys()
kaggleUsername, kaggleKey = keyDict[ userTitle ], keyDict[ keyTitle ]

os.environ[ 'KAGGLE_USERNAME' ] = kaggleUsername
os.environ[ 'KAGGLE_KEY' ] = kaggleKey

api = KaggleApi()
api.authenticate()


In [3]:
# retrieve dataset
datasetOwner = 'lucafrance'
datasetName = 'the-world-factbook-by-cia'
api.dataset_download_files( f'{datasetOwner}/{datasetName}', path="." )

# await download
fTools = FileTools()
datasetFName = None
print( "Waiting for dataset download" )
while True:
    time.sleep( 1 )
    sortedFs = fTools.datesortFiles( currWorkDir, datasetName )
    if len( sortedFs ) == 0: continue
    datasetFName = list( sortedFs )[ 0 ]
    print( f"Latest: {datasetFName}" )
    break

Waiting for dataset download
Latest: the-world-factbook-by-cia.zip


In [4]:
# extract and identify datafiles
origDataDir = f"{currWorkDir}\\data_or"
dsCount = 0
if os.path.exists( origDataDir ): dsCount = len( os.listdir( origDataDir ) )
else: os.makedirs( origDataDir )

unzipped = False
if datasetFName and Path( datasetFName ).suffix == ".zip":
    with zipfile.ZipFile( datasetFName, 'r' ) as zipF:
        zipF.extractall( origDataDir )
    if len( os.listdir() )>dsCount: unzipped = True

dataPaths = [ f"{origDataDir}\\{i}" for i in os.listdir( origDataDir ) ]

if unzipped: print( "Extracted datasets at", dataPaths )

for p in dataPaths:
    if Path( p ).suffix == ".csv": dffBook = pd.read_csv( p ); break

dffBook

Extracted datasets at ['C:\\Users\\romstroller\\PYC\\ciaFactBook\\data_or\\countries.csv', 'C:\\Users\\romstroller\\PYC\\ciaFactBook\\data_or\\countries.json']


,Country,url,Introduction: Background,Geography: Location,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Area - comparative,...,Transportation: Waterways - note 2,Transportation: Waterways - top ten largest natural lakes (by surface area),Transportation: Waterways - note 3,Transportation: Ports and terminals - top twenty container ports as measured by Twenty-Foot Equivalent Units (TEUs) throughput,Transnational Issues: Refugees and internally displaced persons,Transnational Issues: Trafficking in persons - Tier 2 Watch List,Transnational Issues: Trafficking in persons - Tier 3,Transnational Issues: Illicit drugs - cocaine,Transnational Issues: Illicit drugs - opiates,Energy: Electricity access - population without electricity
0,Afghanistan,https://www.cia.gov/the-world-factbook/countri...,Ahmad Shah DURRANI unified the Pashtun tribes ...,"Southern Asia, north and west of Pakistan, eas...","33 00 N, 65 00 E",Asia,"652,230 sq km","652,230 sq km",0 sq km,almost six times the size of Virginia; slightl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akrotiri,https://www.cia.gov/the-world-factbook/countri...,By terms of the 1960 Treaty of Establishment t...,"Eastern Mediterranean, peninsula on the southw...","34 37 N, 32 58 E",Middle East,123 sq km,NaN,NaN,"about 0.7 times the size of Washington, DC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,https://www.cia.gov/the-world-factbook/countri...,Albania declared its independence from the Ott...,"Southeastern Europe, bordering the Adriatic Se...","41 00 N, 20 00 E",Europe,"28,748 sq km","27,398 sq km","1,350 sq km",slightly smaller than Maryland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,https://www.cia.gov/the-world-factbook/countri...,Algeria has known many empires and dynasties s...,"Northern Africa, bordering the Mediterranean S...","28 00 N, 3 00 E",Africa,"2,381,740 sq km","2,381,740 sq km",0 sq km,slightly less than 3.5 times the size of Texas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,American Samoa,https://www.cia.gov/the-world-factbook/countri...,Tutuila was settled by 1000 B.C. and the islan...,"Oceania, group of islands in the South Pacific...","14 20 S, 170 00 W",Oceania,224 sq km,224 sq km,0 sq km,"slightly larger than Washington, DC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,West Bank,https://www.cia.gov/the-world-factbook/countri...,The landlocked West Bank - the larger of the t...,"Middle East, west of Jordan, east of Israel","32 00 N, 35 15 E",Middle East,"5,860 sq km","5,640 sq km",220 sq km,slightly smaller than Delaware,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,World,https://www.cia.gov/the-world-factbook/countri...,"Globally, the 20th century was marked by: (a) ...",NaN,NaN,Physical Map of the World,510.072 million sq km,148.94 million sq km,361.9 million sq km,land area about 16 times the size of the US,...,there are 20 countries without rivers: 3 in Af...,"Caspian Sea (Azerbaijan, Iran, Kazakhstan, Rus...","the deepest lake in the world (1,620 m), and a...","Shanghai (China) - 43,303,000; Singapore (Sing...",the UN High Commissioner for Refugees (UNHCR) ...,"(44 countries) Armenia, Aruba, Azerbaijan, Bar...","(19 countries) Afghanistan, Algeria, Belarus, ...",worldwide coca cultivation in 2020 likely amou...,worldwide illicit opium poppy cultivation prob...,NaN
255,Yemen,https://www.cia.gov/the-world-factbook/countri...,The Kingdom of Yemen (colloquially known as No...,"Middle East, bordering the Arabian Sea, Gulf o...","15 00 N, 48 00 E",Middle East,"527,968 sq km","527,968 sq km",0 sq km,almost four times the size of Alabama; slightl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16 million (2019)
256,Zambia,https://www.cia.gov/the-world-factbook/countri...,Multiple waves of Bantu-speaking groups moved ...,"Southern Africa, east of Angola, south of 

In [5]:
# Analyse cell data for numbers and units

def matchNumbers( df_in, coIdex, patrn ):
    # get any number-pattern match from each row in a list
    return df_in.iloc[ :, coIdex ].str.findall( patrn )


def getRemainder( df_in, coIdex, pattrn ):
    # store non-number remainder of string (potential unit etc)
    return df_in.iloc[ :, coIdex ].str.replace( pattrn, '' )


# MATCHING NUMBERS
# match basic number last to ensure greatest length matched first.
#   capture group             (
#   optional sign             [+-]?
#   1-3 nums                  \d{1,3}
#   non-capture subgroup      (?:
#     comma and 3 nums        ,\d{3}
#     zero or more times      )*
#   non-catpure subgroup      (?:
#     decimal and 1+nums      \.\d+
#     zero or one times       )?
#   OR (alt. to ALL prior)    |
#     0+ nums, dec, 1+nums    \d*\.\d+
#   OR (alt. to ALL prior)    |
#     1+ nums                 \d+


patt = re.compile( r'([+-]?\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d*\.\d+|\d+)' )
dfFbDict = { }  # to collect column data during cleaning
checkTypes = { }
colDex = 1
for colName in dffBook.columns[ colDex: ]:
    origCol = dffBook.iloc[ :, colDex ]
    # get match if string, store if already float, catch unexpected
    if type( origCol[ 0 ] ) == str: colDict = {
        'matchedNums': matchNumbers( dffBook, colDex, patt ),
        'remainder': getRemainder( dffBook, colDex, patt ) }
    elif type( origCol[ 0 ] ) == np.float64: colDict = {
        'matchedNums': origCol.astype( float ), 'remainder': [ ] }
    elif type( origCol[ 0 ] ) != float:
        checkTypes[ type( origCol[ 0 ] ) ] = colDex
        colDict = { 'matchedNums': origCol, 'remainder': [ ] }
    else: colDict = {
        'matchedNums': matchNumbers( dffBook, colDex, patt ),
        'remainder': getRemainder( dffBook, colDex, patt ) }
    colDict[ 'origCol' ] = origCol
    dfFbDict[ colName ] = colDict
    colDex += 1

for i in checkTypes: print( f"unexpected: {i} at {checkTypes[ i ]}" )


In [6]:
# split first match item from list as float to "clean", store else to "split"
def splitFirstOther( matchList ):
    firstVals, splitVals, checkVals = [ ], [ ], [ ]
    for i in range( len( matchList ) ):
        el = matchList[ i ]
        isFilldList = (type( el ) == list) and (len( el )>0)
        if isFilldList:  # remove any thousandcomma to support convert
            firstVals.append( float( ''.join( el[ 0 ].split( ',' ) ) ) )
            splitVals.append( [ v for v in el[ 1: ] ] )
        else:  # check all else are either nan or empty matchlist
            if ((type( el ) == list and len( el )>0) and
                    (type( el ) != list and math.isnan( el ) == False)):
                checkVals.append( el )
            firstVals.append( np.nan )
            splitVals.append( np.nan )
    return firstVals, splitVals, checkVals


for colName in dfFbDict:
    colDict = dfFbDict[ colName ]
    (colDict[ 'clean' ],
     colDict[ 'splitVals' ],
     colDict[ 'checkVals' ]
     ) = splitFirstOther( colDict[ 'matchedNums' ] )

# Raise message if got uncategorized data
for colName in dfFbDict:
    if len( dfFbDict[ colName ][ 'checkVals' ] )>0:
        print( f"Got checkvals for {colName}" )

In [7]:
# dictionary columns to DF, checking is now float
newCols = [ ]
dfFloat = dffBook.iloc[ :, 0 ]  # start with countries
for colName in dfFbDict:
    clean = pd.Series( dfFbDict[ colName ][ 'clean' ] )
    lenFloat = len( [ i for i in clean if type( i ) == float ] )
    if lenFloat>len( clean ) * 0.90:
        newCols.append( colName )
        dfFloat = pd.concat( [ dfFloat, clean ], axis=1 )
    else: print( "col is less than 90% float. Dropping..." )

dfFloat.columns = [ 'Country' ] + newCols
dfFloat


,Country,url,Introduction: Background,Geography: Location,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Area - comparative,...,Transportation: Waterways - note 2,Transportation: Waterways - top ten largest natural lakes (by surface area),Transportation: Waterways - note 3,Transportation: Ports and terminals - top twenty container ports as measured by Twenty-Foot Equivalent Units (TEUs) throughput,Transnational Issues: Refugees and internally displaced persons,Transnational Issues: Trafficking in persons - Tier 2 Watch List,Transnational Issues: Trafficking in persons - Tier 3,Transnational Issues: Illicit drugs - cocaine,Transnational Issues: Illicit drugs - opiates,Energy: Electricity access - population without electricity
0,Afghanistan,NaN,174.0,NaN,33.0,NaN,652230.000,652230.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akrotiri,NaN,196.0,NaN,34.0,NaN,123.000,NaN,NaN,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,NaN,191.0,NaN,41.0,NaN,28748.000,27398.00,1350.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,NaN,3.0,NaN,28.0,NaN,2381740.000,2381740.00,0.0,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,American Samoa,NaN,100.0,NaN,14.0,NaN,224.000,224.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,West Bank,NaN,15.0,NaN,32.0,NaN,5860.000,5640.00,220.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,World,NaN,20.0,NaN,NaN,NaN,510.072,148.94,361.9,16.0,...,20.0,372960.0,1620.0,43303000.0,202.0,44.0,19.0,202.0,265000.0,NaN
255,Yemen,NaN,191.0,NaN,15.0,NaN,527968.000,527968.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
256,Zambia,NaN,188.0,NaN,15.0,NaN,752618.000,743398.00,9220.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Enforce non-nan threshold for rows and columns

def nanThreshold( notNan ):
    # average plus .5 standard deviation (rounded)
    return int( (sum( notNan ) / len( notNan )) + 0.5 * np.std( notNan ) )


def nonNanFromDims( df, dim = 1 ):
    nonNans = [ ]
    for pos in range( 0, df.shape[ dim ] ):
        vals = (df.iloc[ :, pos ].tolist() if dim == 1
                else df.loc[ [ pos ] ].values.tolist()[ 0 ])

        nonNans.append( [ vals, len( [ i for i in vals if (
                type( i ) == float and not math.isnan( i )) ] ) ] )

    _thresh = nanThreshold( [ val for _, val in nonNans ] )
    keepVals = [ val for val, nnul in nonNans if nnul>=_thresh ]
    print( f"non-nan[ {len( keepVals )} ] thr[ {_thresh} ] dim[ {dim} ]" )

    return keepVals

In [9]:
# ROWS
dfRowsClean = pd.DataFrame( nonNanFromDims( dfFloat, dim=0 ) )

# add a columnindex row to track names of kept columns
dfRowsClean.loc[ -1 ] = dfFloat.columns
dfRowsClean.index = dfRowsClean.index + 1
dfRowsClean.sort_index( inplace=True )
dfRowsClean

non-nan[ 125 ] thr[ 255 ] dim[ 0 ]


,0,1,2,3,4,5,6,7,8,9,...,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053
0,Country,url,Introduction: Background,Geography: Location,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Area - comparative,...,Transportation: Waterways - note 2,Transportation: Waterways - top ten largest na...,Transportation: Waterways - note 3,Transportation: Ports and terminals - top twen...,Transnational Issues: Refugees and internally ...,Transnational Issues: Trafficking in persons -...,Transnational Issues: Trafficking in persons -...,Transnational Issues: Illicit drugs - cocaine,Transnational Issues: Illicit drugs - opiates,Energy: Electricity access - population withou...
1,Afghanistan,NaN,174.0,NaN,33.0,NaN,652230.0,652230.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,NaN,191.0,NaN,41.0,NaN,28748.0,27398.0,1350.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,NaN,3.0,NaN,28.0,NaN,2381740.0,2381740.0,0.0,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,NaN,14.0,NaN,12.0,NaN,1246700.0,1246700.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Venezuela,NaN,183.0,NaN,8.0,NaN,912050.0,882050.0,30000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,Vietnam,NaN,10.0,NaN,16.0,NaN,331210.0,310070.0,21140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Yemen,NaN,191.0,NaN,15.0,NaN,527968.0,527968.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
124,Zambia,NaN,188.0,NaN,15.0,NaN,752618.0,743398.0,9220.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# COLS
keepCols = nonNanFromDims( dfRowsClean, dim=1 )

# replace dropped country col, convert to df
keepCols.insert( 0, dfRowsClean.iloc[ :, 0 ].tolist() )
npCols = np.column_stack( keepCols )
dfColsClean = pd.DataFrame( npCols )
dfColsClean

non-nan[ 267 ] thr[ 57 ] dim[ 1 ]


,0,1,2,3,4,5,6,7,8,9,...,258,259,260,261,262,263,264,265,266,267
0,Country,Introduction: Background,Geography: Geographic coordinates,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Land boundaries - total,Geography: Coastline,Geography: Elevation - highest point,Geography: Elevation - lowest point,...,Transportation: Railways - standard gauge,Transportation: Merchant marine - total,Transportation: Merchant marine - by type,Transnational Issues: Refugees and internally ...,Environment: Waste and recycling - municipal s...,Environment: Waste and recycling - percent of ...,Transportation: Railways - narrow gauge,Geography: Maritime claims - exclusive economi...,Economy: Credit ratings - Fitch rating,Military and Security: Military deployments
1,Afghanistan,174.0,33.0,652230.0,652230.0,0.0,5987.0,0.0,7492.0,258.0,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,Albania,191.0,41.0,28748.0,27398.0,1350.0,691.0,362.0,2764.0,0.0,...,677.0,70.0,47.0,1528.0,nan,nan,nan,nan,nan,nan
3,Algeria,3.0,28.0,2381740.0,2381740.0,0.0,6734.0,998.0,2908.0,-40.0,...,2888.0,114.0,1.0,nan,990299.0,8.0,1085.0,nan,nan,nan
4,Angola,14.0,12.0,1246700.0,1246700.0,0.0,5369.0,1600.0,2620.0,0.0,...,nan,54.0,13.0,nan,nan,nan,2729.0,200.0,202.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Venezuela,183.0,8.0,912050.0,882050.0,30000.0,5267.0,2800.0,4978.0,0.0,...,447.0,281.0,4.0,nan,nan,nan,nan,200.0,201.0,nan
122,Vietnam,10.0,16.0,331210.0,310070.0,21140.0,4616.0,3444.0,3144.0,0.0,...,178.0,1926.0,116.0,30581.0,2201169.0,23.0,2169.0,200.0,201.0,nan
123,Yemen,191.0,15.0,527968.0,527968.0,0.0,1601.0,1906.0,3666.0,0.0,...,nan,34.0,2.0,nan,386946.0,8.0,nan,200.0,nan,nan
124,Zambia,188.0,15.0,752618.0,743398.0,9220.0,6043.15,0.0,2330.0,329.0,...,nan,2.0,1.0,nan,nan,nan,3126.0,nan,202.0,925.0
